# Graph Lab

## Header information:

  - Author #1: Luigi Quattrociocchi (quattrl@mcmaster.ca)
  - Author #2: Dennis Fong (fongd1@mcmaster.ca)
  - Gitlab URL: http://gitlab.cas.mcmaster.ca/quattrl/l1-graph-lab
  - Avenue to Learn group name: Graph 42

In [1]:
# import matplotlib.pyplot as plt
# from matplotlib.ticker import MaxNLocator
from csv_reader import *
from graph_builder import build_graph_from_csvs
from metric_extractor import *

In [2]:
G = build_graph_from_csvs("_dataset/london")
metrics = GraphMetrics(G)
print(f"{metrics.num_nodes() = }")
print(f"{metrics.num_edges() = }")
degrees = [metrics.degree(x) for x in range(len(G.adj))]
avg_degree = sum(degrees) / len(degrees)
print(f"{avg_degree = }")

# N = max(degrees) + 1
# fig, ax = plt.subplots(1, 1)
# N, bins, patches = ax.hist(degrees, bins=N, range=(0, N))
# ax.yaxis.set_major_locator(MaxNLocator(integer=True))
# _ = plt.xlabel("Degree")
# _ = plt.ylabel("Number of Nodes")


metrics.num_nodes() = 304
metrics.num_edges() = 406
avg_degree = 2.6710526315789473


In [5]:

# TODO: don't use these functions!
# Make a class to store all information from csvs,
# and create mapping from id to name, and vice versa
def get_name_of_id(name):
    stations = read_csv_contents(f"_dataset/london.stations.csv")
    for station in stations:
        if name == station.name: return station.id
    return -1
def get_id_of_name(u):
    stations = read_csv_contents(f"_dataset/london.stations.csv")
    for station in stations:
        if u == station.id: return station.name
    return ""

n1 = "Hanwell"
n2 = "Stratford"
u = get_name_of_id(n1)
v = get_name_of_id(n2)

# TODO: make an actual benchmark, not this very sad piece of code
import time
a, b = 0, 0
for _ in range(1000):
    t0 = time.time()
    it = Itinerary(u, v)
    it.set_search_strategy(metrics.dijkstras)
    p1 = it.shortest_path(1, 1, 1)
    t1 = time.time()
    a += t1 - t0
    t0 = time.time()
    it = Itinerary(u, v)
    it.set_search_strategy(metrics.astar)
    p2 = it.shortest_path(1, 1, 1)
    t1 = time.time()
    b += t1 - t0
    assert p1 == p2
print(a, b)

0.2537250518798828 0.614661693572998
